# Install

In [ ]:
%pip install tensorflow tf2onnx matplotlib numpy opencv-python scipy

In [ ]:
%pip install --upgrade --force-reinstall tensorflow tf2onnx matplotlib numpy opencv-python scipy mediapipe pandas tqdm scikit-learn 

In [ ]:
import sys
import subprocess

subprocess.check_call([sys.executable, "-m", "pip", "install", "scikit-learn"])

In [ ]:
%pip install --upgrade --force-reinstall tf2onnx

# import and constant

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
import cv2
import string
import os
import shutil
import os
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
import joblib

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [ ]:
dataset_dir = './dataset'
model_name = 'sign_classifier'

# Dataset

In [ ]:
!curl -L -o ./alfabet-bisindo.zip\
  https://www.kaggle.com/api/v1/datasets/download/achmadnoer/alfabet-bisindo

In [ ]:
import zipfile
import os
import platform

zip_path = "./alfabet-bisindo.zip"
extract_dir = "./"

if not os.path.exists(zip_path):
    raise FileNotFoundError(f"❌ File tidak ditemukan: {zip_path}")

print(f"📦 Ekstraksi dataset dari {zip_path} ...")
if platform.system() in ["Windows", "Darwin", "Linux"]:
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print(f"✅ Dataset berhasil diekstrak ke {extract_dir}")
else:
    print(f"⚠️ OS {platform.system()} belum dikenali, pastikan unzip dilakukan manual.")


In [ ]:
import os
import shutil

src_dir = "Citra BISINDO"
dst_dir = "./dataset"

if not os.path.exists(src_dir):
    raise FileNotFoundError(f"❌ Folder sumber tidak ditemukan: {src_dir}")

os.makedirs(dst_dir, exist_ok=True)

print(f"📂 Memindahkan '{src_dir}' ke '{dst_dir}' ...")
shutil.move(src_dir, dst_dir)
print(f"✅ Berhasil dipindahkan ke {dst_dir}")


In [ ]:
!curl -L -o ./indonesian-sign-language-bisindo.zip\
  https://www.kaggle.com/api/v1/datasets/download/agungmrf/indonesian-sign-language-bisindo

In [ ]:
def merge_dataset(src_folder):
    """
    Move files from src_folder into dataset, preserving subfolder (label) structure.
    Example: src_folder/cat -> dataset/cat
             src_folder/dog -> dataset/dog
    """
    # Ensure destination exists
    os.makedirs(dataset_dir, exist_ok=True)

    # Iterate over all label folders in the source
    for label in os.listdir(src_folder):
        label_path_src = os.path.join(src_folder, label)
        label_path_dest = os.path.join(dataset_dir, label)

        # Skip if not a folder
        if not os.path.isdir(label_path_src):
            continue

        # Create label folder in destination if needed
        os.makedirs(label_path_dest, exist_ok=True)

        # Move all files from src → dest
        for filename in os.listdir(label_path_src):
            src = os.path.join(label_path_src, filename)
            dst = os.path.join(label_path_dest, filename)

            # Avoid overwriting files with same name
            if os.path.exists(dst):
                base, ext = os.path.splitext(filename)
                dst = os.path.join(label_path_dest, f"{base}_2{ext}")

            shutil.move(src, dst)

    print(f"✅ Merged '{src_folder}' into '{dataset_dir}' successfully!")

In [ ]:
import zipfile
import os
import platform

zip_path = "./indonesian-sign-language-bisindo.zip"
extract_dir = "./"

if not os.path.exists(zip_path):
    raise FileNotFoundError(f"❌ File tidak ditemukan: {zip_path}")

print(f"📦 Ekstraksi dataset dari {zip_path} ...")
if platform.system() in ["Windows", "Darwin", "Linux"]:
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print(f"✅ Dataset berhasil diekstrak ke {extract_dir}")
else:
    print(f"⚠️ OS {platform.system()} belum dikenali, pastikan unzip dilakukan manual.")


In [ ]:
merge_dataset('./bisindo/images/train')

In [ ]:
merge_dataset('./bisindo/images/val')

In [ ]:
import shutil
import os

folder_path = "./bisindo"

if os.path.exists(folder_path):
    print(f"🗑️ Menghapus folder: {folder_path}")
    shutil.rmtree(folder_path)
    print("✅ Folder berhasil dihapus.")
else:
    print("⚠️ Folder tidak ditemukan, tidak ada yang dihapus.")

In [ ]:
!curl -L -o ./bisindo-dataset.zip\
  https://www.kaggle.com/api/v1/datasets/download/yunitayupratiwi/bisindo-dataset

In [ ]:
import zipfile
import os
import platform

zip_path = "./bisindo-dataset.zip"
extract_dir = "./"

if not os.path.exists(zip_path):
    raise FileNotFoundError(f"❌ File tidak ditemukan: {zip_path}")

print(f"📦 Ekstraksi dataset dari {zip_path} ...")
if platform.system() in ["Windows", "Darwin", "Linux"]:
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print(f"✅ Dataset berhasil diekstrak ke {extract_dir}")
else:
    print(f"⚠️ OS {platform.system()} belum dikenali, pastikan unzip dilakukan manual.")


In [ ]:
def construct_dataset(src_folder):
    """
    Move files from src_folder into dataset, based on the first letter of the filename.
    Example: src_folder/A.66ae97e2-c1e4-11eb-83d3-0008ca6b6d30.jpg -> dataset/A
             src_folder/B.002d8fdf-c1e3-11eb-952a-0008ca6b6d30.jpg -> dataset/B
    """
    # Ensure destination exists
    os.makedirs(dataset_dir, exist_ok=True)

    # Iterate over all file folders in the source
    for filename in os.listdir(src_folder):
        src_file = os.path.join(src_folder, filename)

        # Skip if a folder
        if os.path.isdir(src_file):
            continue
        
        # Skip if not a jpg
        if not src_file.lower().endswith('.jpg'):
            continue
        
        label = filename[0].upper()  # First character as label
        dest = os.path.join(dataset_dir, label)
        # Create label folder in destination if needed
        os.makedirs(dest, exist_ok=True)

        dst_file = os.path.join(dest, filename)

        # Avoid overwriting files with same name
        if os.path.exists(dst_file):
            base, ext = os.path.splitext(filename)
            dst_file = os.path.join(dest, f"{base}_3{ext}")

        shutil.move(src_file, dst_file)

    print(f"successfully!")

In [ ]:
construct_dataset("./BISINDO - Dataset/test")

In [ ]:
construct_dataset("./BISINDO - Dataset/train")

In [ ]:
import shutil
import os

folder_path = "./BISINDO - Dataset"

if os.path.exists(folder_path):
    print(f"🗑️ Menghapus folder: {folder_path}")
    shutil.rmtree(folder_path)
    print("✅ Folder berhasil dihapus.")
else:
    print("⚠️ Folder tidak ditemukan, tidak ada yang dihapus.")

In [ ]:
!curl -L -o ./sign-language-bisindo.zip\
  https://www.kaggle.com/api/v1/datasets/download/bonarsitorus/sign-language-bisindo

In [ ]:
import zipfile
import os
import platform

zip_path = "./sign-language-bisindo.zip"
extract_dir = "./"

if not os.path.exists(zip_path):
    raise FileNotFoundError(f"❌ File tidak ditemukan: {zip_path}")

print(f"📦 Ekstraksi dataset dari {zip_path} ...")
if platform.system() in ["Windows", "Darwin", "Linux"]:
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print(f"✅ Dataset berhasil diekstrak ke {extract_dir}")
else:
    print(f"⚠️ OS {platform.system()} belum dikenali, pastikan unzip dilakukan manual.")


In [ ]:
def merge_dataset_4(src_folder):
    """
    Move files from src_folder into dataset, preserving subfolder (label) structure,
    except folders with '_npy' in their name.
    """
    # Pastikan folder tujuan ada
    os.makedirs(dataset_dir, exist_ok=True)

    # Iterasi semua folder di src_folder
    for label in os.listdir(src_folder):
        # Lewati folder yang mengandung '_npy'
        if "_npy" in label:
            continue

        label_path_src = os.path.join(src_folder, label)
        label_path_dest = os.path.join(dataset_dir, label)

        # Pastikan ini folder
        if not os.path.isdir(label_path_src):
            continue

        # Buat folder di tujuan jika belum ada
        os.makedirs(label_path_dest, exist_ok=True)

        # Pindahkan semua file
        for filename in os.listdir(label_path_src):
            src = os.path.join(label_path_src, filename)
            dst = os.path.join(label_path_dest, filename)

            # Hindari overwrite
            if os.path.exists(dst):
                base, ext = os.path.splitext(filename)
                dst = os.path.join(label_path_dest, f"{base}_4{ext}")

            shutil.move(src, dst)

    print(f"✅ Merged '{src_folder}' into '{dataset_dir}' successfully!")

In [ ]:
merge_dataset_4('./data_tambahan')

In [ ]:
import shutil
import os

folder_path = "./data_tambahan"

if os.path.exists(folder_path):
    print(f"🗑️ Menghapus folder: {folder_path}")
    shutil.rmtree(folder_path)
    print("✅ Folder berhasil dihapus.")
else:
    print("⚠️ Folder tidak ditemukan, tidak ada yang dihapus.")

In [ ]:
!curl -L -o ./indonesian-hand-sign-language-bisindo-dataset.zip\
  https://www.kaggle.com/api/v1/datasets/download/kelsha/indonesian-hand-sign-language-bisindo-dataset

In [ ]:
import zipfile
import os
import platform

zip_path = "./indonesian-hand-sign-language-bisindo-dataset.zip"
extract_dir = "./"

if not os.path.exists(zip_path):
    raise FileNotFoundError(f"❌ File tidak ditemukan: {zip_path}")

print(f"📦 Ekstraksi dataset dari {zip_path} ...")
if platform.system() in ["Windows", "Darwin", "Linux"]:
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print(f"✅ Dataset berhasil diekstrak ke {extract_dir}")
else:
    print(f"⚠️ OS {platform.system()} belum dikenali, pastikan unzip dilakukan manual.")


In [ ]:
def merge_dataset_5(src_folder):
    """
    Move files from src_folder into dataset, preserving subfolder (label) structure.
    """
    # Pastikan folder tujuan ada
    os.makedirs(dataset_dir, exist_ok=True)

    # Iterasi semua folder di src_folder
    for label in os.listdir(src_folder):

        label_path_src = os.path.join(src_folder, label)
        label_path_dest = os.path.join(dataset_dir, label)

        # Pastikan ini folder
        if not os.path.isdir(label_path_src):
            continue

        # Buat folder di tujuan jika belum ada
        os.makedirs(label_path_dest, exist_ok=True)

        # Pindahkan semua file
        for filename in os.listdir(label_path_src):
            src = os.path.join(label_path_src, filename)
            dst = os.path.join(label_path_dest, filename)

            # Hindari overwrite
            if os.path.exists(dst):
                base, ext = os.path.splitext(filename)
                dst = os.path.join(label_path_dest, f"{base}_5{ext}")

            shutil.move(src, dst)

    print(f"✅ Merged '{src_folder}' into '{dataset_dir}' successfully!")

In [ ]:
merge_dataset_5('./dataset_bisindo/train/images')

In [ ]:
merge_dataset_5('./dataset_bisindo/val/images')

In [ ]:
import shutil
import os

folder_path = "./dataset_bisindo"

if os.path.exists(folder_path):
    print(f"🗑️ Menghapus folder: {folder_path}")
    shutil.rmtree(folder_path)
    print("✅ Folder berhasil dihapus.")
else:
    print("⚠️ Folder tidak ditemukan, tidak ada yang dihapus.")

In [ ]:
!curl -L -o ./bisindo-final.zip\
  https://www.kaggle.com/api/v1/datasets/download/skripsiairlangga/bisindo-final

In [ ]:
import zipfile
import os
import platform

zip_path = "./bisindo-final.zip"
extract_dir = "./bisindo_final"

if not os.path.exists(zip_path):
    raise FileNotFoundError(f"❌ File tidak ditemukan: {zip_path}")

# Membuat folder extract_dir jika belum ada
os.makedirs(extract_dir, exist_ok=True)

print(f"📦 Ekstraksi dataset dari {zip_path} ...")
if platform.system() in ["Windows", "Darwin", "Linux"]:
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print(f"✅ Dataset berhasil diekstrak ke {extract_dir}")
else:
    print(f"⚠️ OS {platform.system()} belum dikenali, pastikan unzip dilakukan manual.")


In [ ]:
def merge_dataset_6(src_folder):
    """
    Move files from src_folder into dataset, preserving subfolder (label) structure.
    """
    # Pastikan folder tujuan ada
    os.makedirs(dataset_dir, exist_ok=True)

    # Iterasi semua folder di src_folder
    for label in os.listdir(src_folder):

        label_path_src = os.path.join(src_folder, label)
        label_path_dest = os.path.join(dataset_dir, label)

        # Pastikan ini folder
        if not os.path.isdir(label_path_src):
            continue

        # Buat folder di tujuan jika belum ada
        os.makedirs(label_path_dest, exist_ok=True)

        # Pindahkan semua file
        for filename in os.listdir(label_path_src):
            src = os.path.join(label_path_src, filename)
            dst = os.path.join(label_path_dest, filename)

            # Hindari overwrite
            if os.path.exists(dst):
                base, ext = os.path.splitext(filename)
                dst = os.path.join(label_path_dest, f"{base}_6{ext}")

            shutil.move(src, dst)

    print(f"✅ Merged '{src_folder}' into '{dataset_dir}' successfully!")

In [ ]:
merge_dataset_6('./bisindo_final')

In [ ]:
import shutil
import os

folder_path = "./bisindo_final"

if os.path.exists(folder_path):
    print(f"🗑️ Menghapus folder: {folder_path}")
    shutil.rmtree(folder_path)
    print("✅ Folder berhasil dihapus.")
else:
    print("⚠️ Folder tidak ditemukan, tidak ada yang dihapus.")

In [ ]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=2)
mp_drawing = mp.solutions.drawing_utils

In [ ]:
def extract_hand_landmarks(image_path):
    image = cv2.imread(image_path)
    if image is None:
        return None
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    result = hands.process(image_rgb)
    
    if not result.multi_hand_landmarks:
        return None
    
    # List semua tangan yang terdeteksi
    coords_all = []
    for landmarks in result.multi_hand_landmarks:
        coords = []
        for lm in landmarks.landmark:
            coords.extend([lm.x, lm.y, lm.z])
        coords_all.append(coords)
    
    # Jika hanya 1 tangan, tambahkan 0 agar panjang fitur tetap konsisten
    if len(coords_all) == 1:
        coords_all.append([0.0]*63)  # tangan kosong
    
    # Flatten dua tangan (kanan + kiri)
    features = np.array(coords_all[0] + coords_all[1])
    return features

In [ ]:
data = []
labels = []

for label in sorted(os.listdir(dataset_dir)):
    label_path = os.path.join(dataset_dir, label)
    if not os.path.isdir(label_path):
        continue

    print(f'Processing label: {label}')
    for img_name in tqdm(os.listdir(label_path)):
        img_path = os.path.join(label_path, img_name)
        landmarks = extract_hand_landmarks(img_path)
        if landmarks is not None:
            data.append(landmarks)
            labels.append(label)

In [ ]:
os.makedirs("preprocessing", exist_ok=True)

df = pd.DataFrame(data)
df['label'] = labels

csv_path = os.path.join("preprocessing", "hand_keypoints.csv")
df.to_csv(csv_path, index=False)

print(f"✅ Data dan label berhasil disimpan ke {csv_path}")

# Train

In [4]:
csv_input_path = os.path.join("preprocessing", "hand_keypoints.csv")
df = pd.read_csv(csv_input_path)
print(f"Total data: {len(df)}")
print(df.head())

Total data: 25305
          0         1             2         3         4         5         6  \
0  0.589252  0.788577  7.732578e-08  0.540710  0.797574 -0.010122  0.494010   
1  0.816977  0.762783 -9.139245e-08  0.801779  0.732191  0.003005  0.774102   
2  0.725103  0.491935 -4.427249e-07  0.658864  0.500457 -0.027891  0.584359   
3  0.188371  0.656543 -8.581127e-08  0.283549  0.646443 -0.036480  0.366026   
4  0.113166  0.822599  5.401650e-07  0.195477  0.813338 -0.077608  0.288585   

          7         8         9  ...       117       118       119       120  \
0  0.776065 -0.034902  0.458915  ...  0.259818  0.687959 -0.093514  0.256858   
1  0.710470  0.000877  0.747481  ...  0.627653  0.740314 -0.056922  0.625121   
2  0.476563 -0.053607  0.521293  ...  0.223792  0.330818 -0.103738  0.219856   
3  0.589501 -0.066297  0.421726  ...  0.758943  0.368739 -0.107302  0.773216   
4  0.775300 -0.115059  0.381177  ...  0.871711  0.542666 -0.079876  0.875374   

        121       122     

In [ ]:
le = LabelEncoder()
y = le.fit_transform(df['label'])
X = df.drop('label', axis=1).values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.4),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(y_train.shape[1], activation='softmax')
])

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                    epochs=1000, batch_size=32)

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()


# save model

In [ ]:
import os
import pickle

os.makedirs("models", exist_ok=True)

model_path = os.path.join("models", f"{model_name}.keras")
label_path = os.path.join("models", f"{model_name}_label.pkl")

model.save(model_path)

with open(label_path, "wb") as f:
    pickle.dump(le, f)

print(f"✅ Model disimpan di {model_path}")
print(f"✅ Label encoder disimpan di {label_path}")

In [ ]:
import os
import tensorflow as tf

os.makedirs("models", exist_ok=True)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

tflite_path = os.path.join("models", f"{model_name}.tflite")

with open(tflite_path, "wb") as f:
    f.write(tflite_model)

print(f"✅ Model TFLite disimpan di {tflite_path}")

In [ ]:
import os
import pickle

os.makedirs("models", exist_ok=True)

label_path = os.path.join("models", f"{model_name}_label.pkl")
with open(label_path, "rb") as f:
    le = pickle.load(f)

labels = le.classes_

txt_path = os.path.join("models", "labels.txt")
with open(txt_path, "w") as f:
    for label in labels:
        f.write(label + "\n")

print(f"✅ Labels berhasil disimpan di {txt_path}")